In [4]:
!pip install -r requirements.txt > /dev/null

You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [4]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore

In [5]:
langame_client = LangameClient()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [28]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_general(topics_with_emojis = [
    ("knowledge", "🤔🧠💭📚📖"),
],
    memes_per_topic = 3,
    memes_per_meme=3,
    wikipedia_description = False,
    openai_description = False,
    anti_rate_limit_delay = 1))

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is knowledge, this is a list of starters to ask to other persons:
1.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " What is your favorite color?\n2. What do you think about the \"knowledge economy\"?\n3. Do you like to read books? Which book has changed your life? Why?\n"
    }
  ],
  "created": 1625562181,
  "id": "cmpl-3IbGn8JQxTrue2TP3iSmZ3fXB6CUh",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is knowledge, this is a list of starters to ask to other persons:
1.
try n°0/5
{


In [8]:
# https://emojipedia.org/search
langame_client.generate_save_memes_general(topics_with_emojis = [
    ("philosophy", "🤔🧠💭📚📖"),
    ("mathematic", "🤔🧠💭📚📖📐"),
    ("science", "🤔🧠💭📚📖📐"),
    ("physic", "🤔🧠💭📚📖📐"),
    ("biology", "🤔🧠💭📚📖📐🧬"),
],
    memes_per_topic = 3,
    memes_per_meme=3,
    wikipedia_description = True,
    openai_description = True,
    anti_rate_limit_delay = 1)

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is philosophy, this is a list of starters to ask to other persons:
1.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " What is your philosophy of life?\n2. What it means to be a philosopher?\n3. Do you think that everything has its purpose in life?\n"
    }
  ],
  "created": 1623512934,
  "id": "cmpl-3A0AQfU7c4UPKP9lreBnHqJt2YGT1",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt Below is a long paragraph which describes the topic of 'philosophy' as following:
call_completion with prompt Below is a long paragraph which describes the topic of 'philosophy' as following:
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
   

calling openai_description with prompt Below is a long paragraph which describes the topic of 'mathematics' as following:
call_completion with prompt Below is a long paragraph which describes the topic of 'mathematics' as following:
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\n\"Mathematics is the study of quantity and structure through abstraction, an organized collection of all known mathematical facts. It includes the theories and techniques involved in working with numbers, shapes or formulas."
    }
  ],
  "created": 1623513011,
  "id": "cmpl-3A0BfmoCIqjdkOMARoA5t0P050ZtN",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is mathematics, this is a list of starters to ask to othe

call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is physics, this is a list of starters to ask to other persons:
1.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " \"What is your idea of gravity?\"\n2. \"If you could change the world, would it be more flat or round?\"\n3. \"Have you ever tried to travel at light speed and if so did you get dizzy from the experience\" \u00a0 "
    }
  ],
  "created": 1623513085,
  "id": "cmpl-3A0CrHqsCb1DCkw3koMcGN6dTWgXY",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
calling openai_description with prompt Below is a long paragraph which describes the topic of 'physics' as following:
call_completion with prompt Below is a long paragraph which describes the topic of 'physics' as following:
try n°0/5
{


{
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThe science of biology is the study of life. Biology can be defined as an organismic discipline which concerns itself with the factors that influence living organisms and their behavior, morphology, physiology, reproduction, ecology and evolution. Today biologists use a variety of techniques from molecular genetics to brain imaging in order to help us understand what it means to be alive.[3]\n\nLater on this same page there is presented another paragraph which describes the topic 'physics' as following:\n\nPhysics"
    }
  ],
  "created": 1623513134,
  "id": "cmpl-3A0DeaWSBtmFi4XmUCKqbAOY8STMN",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
call_completion with prompt This is a list of big talk conversation starters. The objective is to propose good conversation starters that create interesting insights between people. Today, the conversation topic is 

[(DatetimeWithNanoseconds(2021, 6, 12, 15, 48, 56, 603449, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa7a02b9f40>),
 (DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 5, 449679, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa76d89c8e0>),
 (DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 16, 311806, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa76d29b5b0>),
 (DatetimeWithNanoseconds(2021, 6, 12, 15, 49, 38, 371022, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa76d29bfa0>),
 (DatetimeWithNanoseconds(2021, 6, 12, 15, 50, 9, 282939, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa76c8ab8e0>),
 (DatetimeWithNanoseconds(2021, 6, 12, 15, 50, 34, 226654, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference at 0x7fa76cb92700>),


In [4]:
# https://emojipedia.org/search
pprint(langame_client.generate_save_memes_ice_break(memes=1, memes_per_meme=3))

call_completion with prompt The goal is to provide good conversation starters that break the ice between people and deeply strengthen their relationships.
Here is a list of deep conversation starters, free from small talks.
1.What is your life's purpose?
2.What insight about life have you acquired, that seems obvious to you but might not be obvious to everyone else?
3.
try n°0/5
{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "What is your biggest regret? How has it affected the way you live your life?\n4.What do you believe about death that most people don't agree with, or would find surprising?\n"
    }
  ],
  "created": 1623922437,
  "id": "cmpl-3BihJFp1pSLsVyGPppJqIX4iFpjZk",
  "model": "davinci:2020-05-03",
  "object": "text_completion"
}
[(DatetimeWithNanoseconds(2021, 6, 17, 9, 33, 59, 964959, tzinfo=datetime.timezone.utc),
  <google.cloud.firestore_v1.document.DocumentReference object at 0x7f0ed18c35b0>),
 (DatetimeW

In [14]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["ice break"])\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .limit(10)\
    .stream():
    pprint(t.reference.parent.parent.get().to_dict())

{'content': 'What do you fantasize about?\n'
            'If you could be absolutely certain of something, what would it '
            'be?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 36, 206000, tzinfo=<UTC>)}
{'content': 'What is the most valuable thought that you have had today?\n'
            'What are your thoughts on religion, God or spirituality?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 34, 120000, tzinfo=<UTC>)}
{'content': 'What kind of person are you when you are your true self?\n'
            'What do you believe is the best in life? What brings you joy and '
            'happiness?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 31, 997000, tzinfo=<UTC>)}
{'content': 'If you could ask God one question, what would it be?\n'
            'What makes you angry or passionate about life?',
 'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 29, 530000, tzinfo=<UTC>)}
{'content': 'What is something you have learned about yoursel

In [60]:
for e in langame_client._firestore_client.collection('langames')\
    .where('done', '==', None)\
    .stream():
        pprint(e.to_dict())

{'channelName': '3d8f9bb0',
 'currentMeme': 0,
 'date': '2021-06-14T14:32:55.812609Z',
 'done': None,
 'initiator': 'htFa1oZEDwUgWbkixfSCqP2jYj12',
 'memes': ['sZDXsUYYpKGPKJLiYHSK',
           '7amMvDemey6vfufUacnR',
           'oBQmqRNVCjP2twdhaY0a'],
 'nextMeme': DatetimeWithNanoseconds(2021, 6, 14, 14, 33, 59, 418000, tzinfo=<UTC>),
 'topics': ['artificial intelligence']}
{'channelName': '8503eeaf',
 'currentMeme': 0,
 'date': '2021-06-14T14:46:34.325771Z',
 'done': None,
 'initiator': 'htFa1oZEDwUgWbkixfSCqP2jYj12',
 'memes': ['oBQmqRNVCjP2twdhaY0a',
           'sZDXsUYYpKGPKJLiYHSK',
           '7amMvDemey6vfufUacnR'],
 'nextMeme': DatetimeWithNanoseconds(2021, 6, 14, 14, 47, 35, 161000, tzinfo=<UTC>),
 'topics': ['artificial intelligence']}
{'channelName': 'a29ba3ac',
 'currentMeme': 0,
 'date': '2021-06-14T14:58:35.702211Z',
 'done': None,
 'initiator': 'htFa1oZEDwUgWbkixfSCqP2jYj12',
 'memes': ['sZDXsUYYpKGPKJLiYHSK',
           '7amMvDemey6vfufUacnR',
           'oBQmqRNVCjP2

In [61]:
selected = []
for tag in langame_client._firestore_client.collection_group(u"tags")\
    .where('topic.content', 'in', ['ice break'])\
    .limit(10)\
    .stream():
    for tag_with_score in tag.reference.parent.parent.collection('tags')\
        .order_by('aggregatedFeedback.general.score', direction=firestore.Query.DESCENDING)\
        .stream():
        
        pprint(tag_in_topics.to_dict())

FailedPrecondition: 400 The query requires a COLLECTION_GROUP_DESC index for collection tags and field aggregatedFeedback.general.score. You can create it here: https://console.firebase.google.com/v1/r/project/langame-86ac4/firestore/indexes?create_exemption=Cmhwcm9qZWN0cy9sYW5nYW1lLTg2YWM0L2RhdGFiYXNlcy8oZGVmYXVsdCkvY29sbGVjdGlvbkdyb3Vwcy90YWdzL2ZpZWxkcy9hZ2dyZWdhdGVkRmVlZGJhY2suZ2VuZXJhbC5zY29yZRACGiQKIGFnZ3JlZ2F0ZWRGZWVkYmFjay5nZW5lcmFsLnNjb3JlEAI 

In [38]:
[e.reference.parent.parent.get().to_dict() for e in selected]

[{'content': 'What kind of person are you when you are your true self?\nWhat do you believe is the best in life? What brings you joy and happiness?',
  'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 31, 997000, tzinfo=<UTC>)},
 {'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 45, 33, 77000, tzinfo=<UTC>),
  'content': 'How would you improve (insert topic) in your local community?\nName a few things that you feel are cool about the generation you grew up in.'},
 {'createdAt': DatetimeWithNanoseconds(2021, 6, 12, 15, 45, 33, 77000, tzinfo=<UTC>),
  'content': 'How would you improve (insert topic) in your local community?\nName a few things that you feel are cool about the generation you grew up in.'},
 {'createdAt': DatetimeWithNanoseconds(2021, 6, 9, 20, 1, 59, 875000, tzinfo=<UTC>),
  'content': 'What’s the last thing you learned?\nWhat TV show is on your bucket list?\nWhat was your favorite childhood game / toy? What would it be today, if you could bring that back?\nWh

In [25]:
import json
memes = []
for m in langame_client._firestore_client.collection(u"memes").stream():
    v = m.to_dict()
    v['createdAt'] = v['createdAt'].timestamp()
    v['id'] = m.id
    v['tags'] = [] 
    for t in m.reference.collection('tags').stream():
        x = t.to_dict()
        x['createdAt'] = x['createdAt'].timestamp()
        x['id'] = t.id
        v['tags'].append(x)
    memes.append(v)

with open('memes.json', 'w') as outfile:
    json.dump({'memes': memes}, outfile, indent=4)


In [9]:
for t in langame_client._firestore_client.collection_group(u"tags").limit(5).stream():
    pprint(t.to_dict())

{'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 32, 330000, tzinfo=<UTC>),
 'origin': {'openai': {'version': 1}}}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 15, 10, 3, 32, 167000, tzinfo=<UTC>),
 'topic': {'content': 'ice break', 'emojis': ['🤔', '😜', '💭']}}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 9, 20, 0, 53, 853000, tzinfo=<UTC>),
 'topic': {'content': 'sciences', 'emojis': ['🤔', '🧠', '💭', '📚', '📖', '📐']}}
{'context': {'content': '"Also known as natural sciences, this field of study '
                        'includes biology, chemistry and physics. These are '
                        'the disciplines most commonly discussed in classrooms '
                        'because they apply directly to everyday life ...',
             'type': 'OPENAI'},
 'createdAt': DatetimeWithNanoseconds(2021, 6, 9, 20, 1, 1, 123000, tzinfo=<UTC>)}
{'createdAt': DatetimeWithNanoseconds(2021, 6, 9, 20, 0, 54, 33000, tzinfo=<UTC>),
 'origin': {'openai': {'version': 1}}}


In [7]:
import json
from datetime import datetime
with open('memes.json', 'r') as f:
    memes = json.load(f)
    pprint(memes)
    
    for m in memes.get('memes'):
        ref = langame_client._firestore_client.collection(u"memes")\
            .doc(m.get('id'))\
            .set({
                'content': m.get('content'),
                'createdAt': datetime.fromtimestamp(m.get('createdAt')),
            }).reference
        for t in memes.get('memes').get('tags'):
            ref.collection('tags').doc(t.get('id'))\
                .set({
                    'createdAt': datetime.fromtimestamp(t.get('createdAt')),
                    'topic': t.get('topic'),
                    'classification': t.get('classification'),
                    'origin': t.get('origin'),
                    'feedback': t.get('feedback'),
                    'context': t.get('context'),
                })
    


{'memes': [{'content': 'What do you think about the concept of truth?\n'
                       'Have you ever thought that our mind has a limit in its '
                       'ability to understand things and situations, or that '
                       'there might be some hidden variables out there we are '
                       'not aware of?\n'
                       'If time travel was possible, would you go back and '
                       'change something in your past? Why/Why Not?',
            'createdAt': 1623767768.9,
            'id': 'U5c8ofxtkNExp6Y2e799',
            'tags': [{'createdAt': 1623767769.006,
                      'id': 'YiyhrSfDHTro4FOQq5E4',
                      'origin': {'openai': {'version': 1}}},
                     {'createdAt': 1623767768.965,
                      'id': 'sXcJj3chLzfiMlHNLEB8',
                      'topic': {'content': 'philosophy',
                                'emojis': ['🤔', '🧠', '💭', '📚', '📖']}}]}]}


In [5]:
# HACK TO SPLIT MEMES
for m in langame_client._firestore_client.collection(u"memes")\
    .limit(1000)\
    .stream():
    splitted = m.to_dict().get('content').split('\n')
    if len(splitted) > 1:
        pprint(m.id)
        tags = []
        for e in m.reference.collection('tags').list_documents():
            tags.append(e.get().to_dict())
        for s in splitted:
            new_m = langame_client._firestore_client.collection('memes').add({'content': s, 'createdAt': m.get('createdAt')})
            for t in tags:
                new_m[1].collection('tags').add(t)
        langame_client._firestore_client.collection('memes').document(m.id).delete()
        
        

'2eWq6FK6KWbOBb82RRiE'
'3TRy37K9IubAfPvx1QtX'
'4m3Tep6Ys3vLGt9kDjDW'
'4p3E8IZUneofaCNztLKL'
'5JtTlSjFNx8pM5bPxKEm'
'6vRMYpKxZcJenzTyteEh'
'7amMvDemey6vfufUacnR'
'8FXdLdwSnh8OlbhJIxTp'
'8WkuCfAqLIMcgYecfFox'
'EJNEkwmj6GlILUlv0HRt'
'FEV9qcBTKdI1sS4RonBQ'
'FcfC8Rd3U2qnrh3GYShX'
'GZMWMp3oqNPsc49PTEtm'
'Ggd5Cq7yoYT25YKEjES0'
'HLIsvJcwO0fGFCYxzUNC'
'HQV7gMJnt102l6kL2RA1'
'HaZLoV84n7K83vpDXFNk'
'IZI7ZTTA6ZqfCQKHyMhw'
'JIN1ZpFLeoC8tFeIEjLF'
'Jl63zR9M0DKquWcHjbRQ'
'KFVKXrP4nQ5wDmOO4RHo'
'KYfY8XTGWnIBTJFNBqlL'
'L8rOXr57YLiVsu2RGUxM'
'LFi4bniTviVVRnglgf03'
'N8gWYYfzAPeIe3dCXUPv'
'NkYnTEL0ym090xqGEgZF'
'OFs2j2mu76v3bXj5cBVi'
'OwrxFxZgMrZQSiiG7LmF'
'Q2LRKnynWvorxIj7RfQY'
'QEhm7sUDhXTwyvc6gM96'
'RMKtAFz9oTzDUiZspQtZ'
'RyXhlfJqP4qHqOk9UwIy'
'S7KwRJURquQphTa35fKx'
'SyXWFj56lsWybEqhpEnW'
'Tut46VEciJ6A1WzD1WQI'
'Ux8uZCbAaPEXzCmjYnmo'
'Y4D6tYqy9Rt4yD85yX98'
'YbOtj57O5oWHzlCkfHpp'
'Yn0rnNHmIGh4A48pUTXZ'
'ZKXNtGiFtnKqloX9F4BB'
'cHPYgvHMb5Pd3E8vDcXO'
'cKwMAOK6NqNapaLAJ9GI'
'cb6S0JImB4Y5hmErbwrf'
'dvfQXwJbZt

In [3]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .limit(10000)\
    .stream():
        m = t.reference.parent.parent.get()
        if not m.exists:
            pprint(langame_client._firestore_client.collection('tags').document(t.id).delete())
            pprint(langame_client._firestore_client.collection('memes').document(m.id).delete())

DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 22, 433738, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 22, 433738, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 22, 738225, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 22, 738225, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 23, 156126, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 23, 411099, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 110331, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 110331, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 589573, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 589573, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 933037, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 52, 27, 933037

DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 1, 484278, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 1, 484278, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 1, 872870, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 1, 872870, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 2, 264362, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 2, 425106, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 2, 643716, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 2, 643716, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 3, 28919, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 3, 28919, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 3, 421198, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 3, 421198, tzinfo=datet

DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 36, 820452, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 36, 820452, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 37, 221307, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 37, 418730, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 39, 432320, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 39, 432320, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 39, 737343, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 39, 737343, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 40, 113927, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 40, 113927, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 40, 490017, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 53, 40, 490017

DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 19, 900723, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 19, 900723, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 20, 287754, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 20, 433941, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 24, 32248, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 24, 32248, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 24, 428755, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 24, 428755, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 27, 848673, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 27, 848673, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 28, 233392, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 54, 28, 424870, 

DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 18, 466295, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 18, 466295, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 19, 925614, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 19, 925614, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 20, 298495, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 20, 445205, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 26, 437448, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 26, 437448, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 26, 780921, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 26, 780921, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 27, 167108, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 55, 27, 167108

DatetimeWithNanoseconds(2021, 6, 18, 9, 56, 4, 454004, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 18, 9, 56, 4, 454004, tzinfo=datetime.timezone.utc)


KeyboardInterrupt: 

In [15]:
for t in langame_client._firestore_client.collection(u"langames")\
    .list_documents():
        pprint(t.delete())
        for p in t.collection("players").stream():
            pprint(p.reference.delete())

DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 40, 969025, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 41, 311615, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 41, 493471, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 41, 612413, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 41, 962639, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 42, 56079, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 42, 420671, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 42, 469542, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 43, 559, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 43, 170431, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 43, 317218, tzinfo=datetime.timezone.utc)
DatetimeWithNanoseconds(2021, 6, 27, 13, 33, 43

In [23]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where("topic.content", "==", "sciences")\
    .get():
        pprint(t.reference.update({
            "topic.content": "science"
        }))


update_time {
  seconds: 1625559747
  nanos: 424223000
}

update_time {
  seconds: 1625559747
  nanos: 596319000
}

update_time {
  seconds: 1625559747
  nanos: 742143000
}

update_time {
  seconds: 1625559747
  nanos: 889551000
}

update_time {
  seconds: 1625559748
  nanos: 23775000
}

update_time {
  seconds: 1625559748
  nanos: 164863000
}

update_time {
  seconds: 1625559748
  nanos: 300911000
}

update_time {
  seconds: 1625559748
  nanos: 433471000
}

update_time {
  seconds: 1625559748
  nanos: 567439000
}

update_time {
  seconds: 1625559748
  nanos: 721039000
}

update_time {
  seconds: 1625559748
  nanos: 868527000
}

update_time {
  seconds: 1625559749
  nanos: 10783000
}

update_time {
  seconds: 1625559749
  nanos: 178767000
}

update_time {
  seconds: 1625559749
  nanos: 336143000
}

update_time {
  seconds: 1625559749
  nanos: 504031000
}

update_time {
  seconds: 1625559749
  nanos: 654191000
}

update_time {
  seconds: 1625559749
  nanos: 787535000
}

update_time {
  